In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article
# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os

import ollama
from tqdm import tqdm
import time
import re
import sys
import pandas as pd

module_path = '../core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KeywordExtractor import KeywordExtractor
from TripletGenerator import TripletGenerator

/home/niamatzawad/miniconda3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows


In [3]:
class KG:
    def __init__(self,):
        self.triplets = []

    def add_triplets(self, triplets : List):
        self.triplets.extend(triplets)


In [4]:
def retrieve_text_from_filepath(filepath):
    with open(filepath,'r') as f:
        t = f.read()
        return t


In [5]:
%%time
# folder_path = "/home/niamatzawad/niamatzawad/Datasets/UTDBox/demoNY/unsplitted"
folder_path = "/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4"

s_demo = SemanticSimilarity()
kwe_demo = KeywordExtractor()
kg_demo = KG()

folder_objects = os.listdir(folder_path)
for i, filename in enumerate(folder_objects):
    if os.path.isfile(os.path.join(folder_path, filename)):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            text_splitted = [text[i * 1900:(i+1) *1900] for i in range(len(text)//1900)]
            start_text_time = time.time()
            for text_s in tqdm(text_splitted):
                start_subtext_time = time.time()
                try:
                    named_entities = kwe_demo.extract_named_entities(text_s)
                except Exception as e:
                    print(e)
                    continue
                s_demo.generate_reference(named_entities)
                t = TripletGenerator(text_s, s_demo, kwe_demo, os.path.join(folder_path, filename), named_entities=named_entities)
                triplets = t.get_triplets()
                kg_demo.add_triplets(triplets)
                print(f"Time to create triplets for current subtext - {time.time() - start_subtext_time}")
                
        print(f"Finished processing {i+1}/ {len(folder_objects)} files")
        print(f"Total time taken for current text - {time.time() - start_text_time}")
        print("---")
        


  0%|                                                     | 0/2 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'model': 'mistral', 'created_at': '2024-05-24T22:02:18.470519863Z', 'response': ' Triplet 1: (The Ritz-Carlton Jakarta, is located near, The sister JW Marriott Hotel)\nText: "It is located in city center of Jakarta, near Mega Kuningan, adjacent to the sister JW Marriott Hotel."\n\nTriplet 2: (The Oberoi Group, operates, The Ritz-Carlton Jakarta)\nText: "It is operated by The Ritz-Carlton Hotel Company, which is a part of The Oberoi Group."\n\nTriplet 3: (Nakuul Mehta, portrays, Shivaay)\nText: "Nakuul Mehta respectively portrays Shivaay"\n\nTriplets 4 and 5: None\nThere\'s no clear action relation for the given entities in these texts.\n\nTriplet 6: (The Eppley Hotel Company, contributed, local citizens)\nText: "local citizens contributed $120,000 to ensure the successful completion of this seven-story hotel"\n\nTriplet 7: (The Sheraton Corporation, bought, The Tallcorn)\nText: "The Tallcorn was sold from the Eppley chain to the Sheraton Corporation"\n\nTriplet 8: (Marshalltown, is li

 50%|██████████████████████▌                      | 1/2 [00:07<00:07,  7.56s/it]

Time to create triplets for current subtext - 7.563148021697998


100%|█████████████████████████████████████████████| 2/2 [00:14<00:00,  7.17s/it]


...

Time to create triplets for current subtext - 8.307557821273804
Finished processing 3/ 4 files
Total time taken for current text - 8.31067442893982
---


 50%|██████████████████████▌                      | 1/2 [00:04<00:04,  4.15s/it]

{'model': 'mistral', 'created_at': '2024-05-24T22:03:08.59048965Z', 'response': ' Triplet 1: (Radio City, was started in, Mumbai)\nText: "It was started in Mumbai in 2004."\n\nTriplet 2: (Albanian Football Federation, was founded on, June 6, 1930)\nText: "Albanian National Team was founded on June 6, 1930,"\n\nTriplet 3: (Echosmith, signed to, Warner Bros. Records)\nText: "They started first as a quartet of siblings, the band currently consists of Sydney, Noah and Graham Sierota, following the departure of eldest sibling Jamie in late 2016. Echosmith started first as \'Ready Set Go!\' until they signed to Warner Bros. Records in May 2012."', 'done': True, 'done_reason': 'stop', 'context': [733, 16289, 28793, 28705, 10290, 28747, 28777, 5067, 272, 19810, 5936, 19036, 2990, 647, 464, 507, 515, 647, 464, 28719, 2915, 1585, 647, 464, 28726, 980, 282, 19099, 647, 464, 28714, 1384, 3306, 647, 464, 1095, 882, 5365, 647, 464, 279, 647, 464, 5942, 826, 20882, 647, 464, 1839, 1520, 28710, 647, 4

100%|█████████████████████████████████████████████| 2/2 [00:09<00:00,  4.84s/it]

Time to create triplets for current subtext - 5.516457557678223
Finished processing 4/ 4 files
Total time taken for current text - 9.675930500030518
---
CPU times: user 56.3 s, sys: 5.31 s, total: 1min 1s
Wall time: 1min 4s


In [ ]:
# def get_subjects_from_triplets(triplets):
    return list(set([i[0] for i in triplets]))
get_subjects_from_triplets(kg_demo.triplets)

In [7]:
kg_demo.triplets

[]

In [8]:
# np.save('disambiguated_reference_list_hotpot_100.npy', s.disambiguated_reference_list)    # .npy extension is added if not given
# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

In [9]:
# np.save('embedded_reference_list_hotpot_100.npy', s.embedded_reference)    # .npy extension is added if not given
# embedded_reference = np.load('embedded_reference.npy')

In [10]:
s_demo.disambiguated_reference_list

[]

In [11]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

In [14]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

triplets = kg_demo.triplets

# Create the nodes and relationships
for t in triplets:
    # Create the subject node
    subject, predicate, obj = t[0],t[1],t[2]
    context = None
    if len(t) == 4:
        context = t[3]
    
    try:
        n = graph.run(f"MATCH (n:{subject}) RETURN n;").data()
    except Exception as e:
        print(f"Problem with subject - {subject}")
        continue

    if n:
        subject_node = n[0]["n"]
    else:
        subject_node = Node(subject, id=subject)
        graph.create(subject_node)

    if predicate == "filepath":
        # obj = re.sub("NYT Files", "'NYT Files'",obj)
        obj = "```"+obj+"```"
        context = retrieve_text_from_filepath(obj[3:-3])
        n = graph.run(f"MATCH (n:{obj}) RETURN n;").data()
        if n:
            # print(n[0]["n"])
            object_node = n[0]["n"]
        else:
            # obj = re.sub("'NYT Files'", "NYT Files",obj)[1:-1]
            
            obj = obj[2:-2]
            object_node = Node(obj, id=obj)
            graph.create(object_node)
    else:
        try:
            n = graph.run(f"MATCH (n:{obj}) RETURN n;").data() #I had the assumption that subjects are being linked to wrong entities. Seems that is not the case as it exactly matches object if it exists and not the most similar object
        except Exception as e:
            print(f"Problem with object - {obj}")
            continue
            
        if n:
            object_node = n[0]["n"]
        else:
            object_node = Node(obj, id=obj, text=context)
            graph.create(object_node)

    relationship = Relationship(subject_node, predicate, object_node, text=context)
    graph.merge(relationship)

print("Triplets added to the Neo4j database.")

Problem with object - 27_june_2016
Problem with object - 1989
Problem with object - 1990
Problem with object - 1932
Triplets added to the Neo4j database.


In [15]:
s = SemanticSimilarity()
s.disambiguated_reference_list

[]

In [14]:
np.save('disambiguated_reference_list_kg_demo_hotpot_100.npy', s_demo.disambiguated_reference_list)    # .npy extension is added if not given
np.save('embedded_reference_kg_demo_hotpot_100.npy', s_demo.embedded_reference)    # .npy extension is added if not given


In [34]:
for i in kg_demo.triplets:
    if i[0] == "dua_lipa":
        print(i)
        break

('dua_lipa', 'wrote', 'dance')


In [28]:
kg_demo.triplets

[('kings_county_museum',
  'was_agreed_to_have_space_for',
  'new_brunswick_centennial_administration',
  'An agreement was made between the Kings County Historical Society, the Municipality of Kings County, and the New Brunswick Centennial Administration to have space for a Museum in the new County Centennial Building in Hampton.'),
 ('kings_county_museum',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_100/row_98.txt'),
 ('museum',
  'is_open_to_the_public_for_tours_of_the_exhibit',
  'summertime',
  'The museum is open in the summertime and certain hours in the wintertime to the public for tours of the exhibit and the old gaol.'),
 ('museum',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_100/row_98.txt'),
 ('museum',
  'presents_exhibitions_and_programming_about_natural_history',
  'to_the_public',
  'Fernbank Museum has a mission to encourage a greater appreciation of the planet and its people, and presents exhibitions and programm

In [11]:
triples = []
for k in kg_demo.triplets:
    t = list(k)
    if len(t) == 3:
        t = t + [""]
    if len(t) > 4:
        continue
    triples.append(t)

